# Парсинг skillbox

In [12]:
import pandas as pd
import requests
import json
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math

from IPython.display import clear_output
from bs4 import BeautifulSoup
from time import sleep

In [13]:
HOST   = 'https://hh.ru'
FILTER = '/search/vacancy?st=searchVacancy&text=&specialization=1.89&specialization=1.221&specialization=1.82&specialization=1.110&specialization=1.25&specialization=1.270&specialization=1.10&specialization=1.9&salary=&currency_code=RUR&experience=doesNotMatter&employment=full&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&L_save_area=true'

HEADERS = requests.utils.default_headers()
HEADERS.update({'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
                'accept':'*/*'})
GLOBAL_SUM = 1

## Функция get_html_page возвращает html документ, по данному URL

In [14]:
def get_html_page(url, n_attempts=5, t_sleep=1):
    
    while n_attempts:
        try:
            r = requests.get(url, headers=HEADERS)
            
            if r.status_code != 200:
                return None
            
            html = r.text
            r.close()
            break
        
        except requests.exceptions.RequestException as e:
            n_attempts -= 1
            print(f'Sleep for {t_sleep} sec; {n_attempts} left')
            
            sleep(t_sleep)
            continue
            
    return html

## Функция cleaner производит очистку текста от ненужных символов

In [15]:
def cleaner(string):
    string = re.sub('\n+|\t+|\r+', '', string)
    while string[0] in ' ,.':
        string = string[1:]
    
    while string[-1] == ' ':
        string = string[:-1]
    
    string = re.sub('\ +', ' ', string)
    string = re.sub('\. ', '.', string)
    string = re.sub(' \.', '.', string)
    string = re.sub(', ', ',', string)
    string = re.sub(' ,', ',', string)
    
    return string

## Функция find_all_links возвращает все ссылки, полученные из html документа

In [16]:
def find_all_links(file):
    f = open(file)
    f = f.read()
    soup = BeautifulSoup(f, 'html.parser')
    list_links = list()
    for key in soup.find_all('a'):
        list_links.append(key.get('href'))
    return list_links

## Функция parse_link принимает на вход ссылку на курс и возращает список умений, приобертаемый пользователем, после освоения этого курса

In [17]:
def parse_link(link):
    text = list()
    f = get_html_page(link)
    if not f:
        return None
    soup = BeautifulSoup(f, 'html.parser')
    soup = soup.find('section', attrs={'program block block--default'})
    return (link, cleaner(soup.text.lower()))

## Функция parse_skill_box принимает на вход ссылки на курсы, и заполняет DataFrame навыками, получаемые после прохождения курсов

In [18]:
def parse_skill_box(list_links):
    d = list()
    k = int()
    for link in list_links:
        x = parse_link(link)
        if x:
            d.append(x)
        else:
            k+=1
    return pd.DataFrame(d, columns=('link', 'text'))

In [20]:
df = parse_skill_box(find_all_links('skillbox.html'))

In [21]:
df

,link,text
0,https://skillbox.ru/course/profession-graphdes...,программа вас ждут 6 блоков с разным уровнем с...
1,https://skillbox.ru/course/3d-generalist/,программа вас ждут 7 курсов с разным уровнем с...
2,https://skillbox.ru/course/profession-marketolog/,"программа вас ждут онлайн-лекции и задания,вып..."
3,https://skillbox.ru/course/profession-data-sci...,"программа вас ждут 9 курсов,онлайн-лекции и пр..."
4,https://skillbox.ru/course/profession-business...,программа вас ждут онлайн-лекции и практически...
...,...,...
157,https://skillbox.ru/course/profession-offline-...,программа вас ждут онлайн-лекции и практически...
158,https://skillbox.ru/course/profession-hr-busin...,программа вас ждут онлайн-лекции и практически...
159,https://skillbox.ru/course/profession-sports-j...,программа вас ждут онлайн-лекции и практически...
160,https://skillbox.ru/course/profession-investme...,программа вас ждут онлайн-лекции и домашние за...


### Сохраняем полученные данные

In [22]:
df.to_csv('../Data/skillbox.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/skillbox.csv'